In [1]:
import numpy as np
import torch
import torch_geometric.nn.norm.batch_norm

from helper import *
from gnn import GNN
import random

In [7]:
julia.install()


Precompiling PyCall...
Precompiling PyCall... DONE
PyCall is installed and built successfully.


The State of the nth node is expressed by 4 real scalars:

v_n -> the voltage at the node
delta_n -> the voltage angle at the node (relative to the slack bus)
p_n -> the net active power flowing into the node
q_n -> the net reactive power flowing into the node


The physical characteristics of the network are described by the power flow equations:

p = P(v, delta, W)
q = Q(v, delta, W)

-> Relate local net power generation with the global state
-> Depends on the topology W of the grid

Electrical grid => Weighted Graph

Nodes in the graph produce/consume power

Edges represent electrical connections between nodes

State Matrix X element of  R(N x 4) => graph signal with 4 features
    => Each row is the state of the corresponding Node

Adjacency Matrix A => sparse matrix to represent the connections of each node, element of R(N x N), Aij = 1 if node i is connected to node j


We use the GNN as a mode phi(X, A, H)

We want to imitate the OPF solution p*
-> We want to minimize a loss L over a dataset T = {{X, p*}}

Objective Function:min arg H of sum over T L(p*,phi(X, A, H)) and we use L = Mean Squared error

Once GNN model phi is trained, we do not need the costly p* from pandapower to make predictions

Input Data X - R^(Nx4): Uniformly sample p_ref and q_ref of each load L with P_L ~ Uniform(0.9 * p_ref, 1.1 * p_ref) and Q_L ~ Uniform(0.9 * q_ref, 1.1 * q_ref)

Pseudocode for X and y in supervised learning:
for each P_L and Q_L:
    Create X with sub-optimal DCOPF results
    Create y with Pandapower calculating p* ACOPF with IPOPT


Spatio-Temporal GNN -> superposition of a gnn with spatial info and a temporal layer (Temporal Conv,LSTM etc.) ??

Bus => Node in GNN








In [2]:
# get lists of simbench codes
all_simbench_codes = sb.collect_all_simbench_codes()
all_simbench_codes

['1-complete_data-mixed-all-0-sw',
 '1-complete_data-mixed-all-1-sw',
 '1-complete_data-mixed-all-2-sw',
 '1-EHVHVMVLV-mixed-all-0-sw',
 '1-EHVHVMVLV-mixed-all-1-sw',
 '1-EHVHVMVLV-mixed-all-2-sw',
 '1-EHVHV-mixed-all-0-sw',
 '1-EHVHV-mixed-all-0-no_sw',
 '1-EHVHV-mixed-all-1-sw',
 '1-EHVHV-mixed-all-1-no_sw',
 '1-EHVHV-mixed-all-2-sw',
 '1-EHVHV-mixed-all-2-no_sw',
 '1-EHVHV-mixed-1-0-sw',
 '1-EHVHV-mixed-1-0-no_sw',
 '1-EHVHV-mixed-1-1-sw',
 '1-EHVHV-mixed-1-1-no_sw',
 '1-EHVHV-mixed-1-2-sw',
 '1-EHVHV-mixed-1-2-no_sw',
 '1-EHVHV-mixed-2-0-sw',
 '1-EHVHV-mixed-2-0-no_sw',
 '1-EHVHV-mixed-2-1-sw',
 '1-EHVHV-mixed-2-1-no_sw',
 '1-EHVHV-mixed-2-2-sw',
 '1-EHVHV-mixed-2-2-no_sw',
 '1-EHV-mixed--0-sw',
 '1-EHV-mixed--0-no_sw',
 '1-EHV-mixed--1-sw',
 '1-EHV-mixed--1-no_sw',
 '1-EHV-mixed--2-sw',
 '1-EHV-mixed--2-no_sw',
 '1-HVMV-mixed-all-0-sw',
 '1-HVMV-mixed-all-0-no_sw',
 '1-HVMV-mixed-all-1-sw',
 '1-HVMV-mixed-all-1-no_sw',
 '1-HVMV-mixed-all-2-sw',
 '1-HVMV-mixed-all-2-no_sw',
 '1-HVM

In [3]:
net = sb.get_simbench_net('1-complete_data-mixed-all-0-sw')

In [ ]:
grids_available = []
for nw_name in all_simbench_codes:
        net = sb.get_simbench_net(nw_name)
        print("Trying Network named " + nw_name + "...")

        #dict_probs = pp.diagnostic(net,report_style='None')
        #for bus_num in dict_probs['multiple_voltage_controlling_elements_per_bus']['buses_with_gens_and_ext_grids']:
        #    net.gen = net.gen.drop(net.gen[net.gen.bus == bus_num].index)

        #OPERATIONAL CONSTRAINTS

        #Set upper and lower limits of active-reactive powers of loads
        min_p_mw_val, max_p_mw_val, min_q_mvar_val, max_q_mvar_val = [], [], [], []
        p_mw = list(net.load.p_mw.values)
        q_mvar = list(net.load.q_mvar.values)

        for i in range(len(p_mw)):
            min_p_mw_val.append(p_mw[i])
            max_p_mw_val.append(p_mw[i])
            min_q_mvar_val.append(q_mvar[i])
            max_q_mvar_val.append(q_mvar[i])

        net.load.min_p_mw = min_p_mw_val
        net.load.max_p_mw = max_p_mw_val
        net.load.min_q_mvar = min_q_mvar_val
        net.load.max_q_mvar = max_q_mvar_val

        #Replace all ext_grids but the first one with generators and set the generators to slack= false
        ext_grids = [i for i in range(1,len(net.ext_grid.name.values))]
        pp.replace_ext_grid_by_gen(net,ext_grids=ext_grids, slack=False)

        #TODO: reactive power limits for gens?


        #TODO: reactive power limits for sgens?



        #NETWORK CONSTRAINTS

        #Maximize the branch limits

        #max_i_ka = list(net.line.max_i_ka.values)

        #for i in range(len(max_i_ka)):
        # max_i_ka[i] = max(max_i_ka)



        #Maximize line loading percents
        max_loading_percent = list(net.line.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.line.max_loading_percent = max_loading_percent

        #Maximize trafo loading percent
        max_loading_percent = list(net.trafo.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.trafo.max_loading_percent = max_loading_percent

        #Maximize trafo3w loading percent
        max_loading_percent = list(net.trafo3w.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.trafo3w.max_loading_percent = max_loading_percent

        #Cost assignment
        pp.create_pwl_costs(net, [i for i in range(len(net.gen.name.values))],et="gen", points=[[[0, 20, 1], [20, 30, 2]] for _ in range(len(net.gen.name.values))])
        pp.create_pwl_costs(net, [i for i in range(len(net.sgen.name.values))],et="sgen", points=[[[0, 20, 0.25], [20, 30, 0.5]] for _ in range(len(net.sgen.name.values))])
        pp.create_pwl_costs(net, [i for i in range(len(net.ext_grid.name.values))],et="ext_grid", points=[[[0, 20, 2], [20, 30, 5]] for _ in range(len(net.ext_grid.name.values))])

        try:
            pp.runpm_dc_opf(net) # Run DCOPP
        except pp.OPFNotConverged:
            text = "DC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ".SKIPPING THIS DATASET."
            print(text)
            continue
        print("GRID NAMED "+nw_name+" CONVERGES FOR DCOPF")
        grids_available.append(nw_name)


In [158]:
for nw_name in grids_available:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) # dcopp on all grids directly

1-HVMV-mixed-all-0-sw: Number of Buses = 1942
1-HVMV-mixed-all-0-no_sw: Number of Buses = 1665
1-HVMV-mixed-1.105-0-sw: Number of Buses = 401
1-HVMV-mixed-1.105-0-no_sw: Number of Buses = 158
1-HVMV-mixed-2.102-0-sw: Number of Buses = 421
1-HVMV-mixed-2.102-0-no_sw: Number of Buses = 178
1-HVMV-mixed-4.101-0-sw: Number of Buses = 411
1-HVMV-mixed-4.101-0-no_sw: Number of Buses = 166
1-HVMV-urban-all-0-sw: Number of Buses = 1791
1-HVMV-urban-all-0-no_sw: Number of Buses = 1470
1-HVMV-urban-2.203-0-sw: Number of Buses = 487
1-HVMV-urban-2.203-0-no_sw: Number of Buses = 196
1-HVMV-urban-3.201-0-sw: Number of Buses = 514
1-HVMV-urban-3.201-0-no_sw: Number of Buses = 217
1-HVMV-urban-4.201-0-sw: Number of Buses = 477
1-HVMV-urban-4.201-0-no_sw: Number of Buses = 184
1-HV-mixed--0-sw: Number of Buses = 306
1-HV-mixed--0-no_sw: Number of Buses = 64
1-HV-urban--0-sw: Number of Buses = 372
1-HV-urban--0-no_sw: Number of Buses = 82
1-MVLV-rural-all-0-sw: Number of Buses = 5479
1-MVLV-rural-all-0

In [ ]:
grids_ready = []
for nw_name in all_simbench_codes[6:]:
    net = sb.get_simbench_net(nw_name)
    print("Trying Network named " + nw_name + "...")

    #OPERATIONAL CONSTRAINTS

    #Set upper and lower limits of active-reactive powers of loads
    min_p_mw_val, max_p_mw_val, min_q_mvar_val, max_q_mvar_val = [], [], [], []
    p_mw = list(net.load.p_mw.values)
    q_mvar = list(net.load.q_mvar.values)

    for i in range(len(p_mw)):
        min_p_mw_val.append(p_mw[i])
        max_p_mw_val.append(p_mw[i])
        min_q_mvar_val.append(q_mvar[i])
        max_q_mvar_val.append(q_mvar[i])

    net.load.min_p_mw = min_p_mw_val
    net.load.max_p_mw = max_p_mw_val
    net.load.min_q_mvar = min_q_mvar_val
    net.load.max_q_mvar = max_q_mvar_val

    #Replace all ext_grids but the first one with generators and set the generators to slack= false
    ext_grids = [i for i in range(1,len(net.ext_grid.name.values))]
    pp.replace_ext_grid_by_gen(net,ext_grids=ext_grids, slack=False)

    #TODO: reactive power limits for gens?


    #TODO: reactive power limits for sgens?



    #NETWORK CONSTRAINTS

    #Maximize the branch limits

    #max_i_ka = list(net.line.max_i_ka.values)

    #for i in range(len(max_i_ka)):
    # max_i_ka[i] = max(max_i_ka)



    #Maximize line loading percents
    max_loading_percent = list(net.line.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.line.max_loading_percent = max_loading_percent

    #Maximize trafo loading percent
    max_loading_percent = list(net.trafo.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.trafo.max_loading_percent = max_loading_percent

    #Maximize trafo3w loading percent
    max_loading_percent = list(net.trafo3w.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.trafo3w.max_loading_percent = max_loading_percent

    #Cost assignment
    pp.create_pwl_costs(net, [i for i in range(len(net.gen.name.values))],et="gen", points=[[[0, 20, 1], [20, 30, 2]] for _ in range(len(net.gen.name.values))])
    pp.create_pwl_costs(net, [i for i in range(len(net.sgen.name.values))],et="sgen", points=[[[0, 20, 0.25], [20, 30, 0.5]] for _ in range(len(net.sgen.name.values))])
    pp.create_pwl_costs(net, [i for i in range(len(net.ext_grid.name.values))],et="ext_grid", points=[[[0, 20, 2], [20, 30, 5]] for _ in range(len(net.ext_grid.name.values))])

    #ac_converged = True

    #start_vec_name = ""
    #for init in ["pf", "flat", "results"]:
    #    try:
    #        pp.runopp(net, init=init)  # Calculate ACOPF with IPFOPT
    #    except pp.OPFNotConverged:
    #        if init == "results":
    #            text = "AC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ". SKIPPING THIS GRID."
    #            print(text)
    #            break
    #        continue
    #    start_vec_name = init
    #    ac_converged = True
    #    break
    #if ac_converged:
    #    print("GRID NAMED "+nw_name+" CONVERGES FOR ACOPF" + "WITH THE START VECTOR OPTION " + start_vec_name + ".")


    try:
        pp.runpm_ac_opf(net) # Run DCOPP
    except pp.OPFNotConverged:
        text = "AC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ".SKIPPING THIS DATASET."
        print(text)
        continue
    print("GRID NAMED "+nw_name+" CONVERGES FOR ACOPF" + "WITH THE START VECTOR OPTION " + ".")
    grids_ready.append(nw_name)


In [4]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #julia acopf on all grids directly

1-HV-mixed--0-sw: Number of Buses = 306
1-HV-mixed--0-no_sw: Number of Buses = 64
1-HV-urban--0-sw: Number of Buses = 372
1-HV-urban--0-no_sw: Number of Buses = 82


In [6]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #pp acopf on all grids directly

1-HV-mixed--0-sw: Number of Buses = 306
1-HV-mixed--0-no_sw: Number of Buses = 64
1-HV-mixed--1-sw: Number of Buses = 355
1-HV-mixed--1-no_sw: Number of Buses = 94
1-HV-urban--0-sw: Number of Buses = 372
1-HV-urban--0-no_sw: Number of Buses = 82
1-HV-urban--1-sw: Number of Buses = 402
1-HV-urban--1-no_sw: Number of Buses = 100
1-MV-semiurb--0-sw: Number of Buses = 117
1-MV-semiurb--0-no_sw: Number of Buses = 115
1-MV-comm--0-sw: Number of Buses = 107
1-MV-comm--0-no_sw: Number of Buses = 103


In [163]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #julia acopf

1-HV-mixed--0-sw: Number of Buses = 306
1-HV-mixed--0-no_sw: Number of Buses = 64
1-HV-urban--0-sw: Number of Buses = 372
1-HV-urban--0-no_sw: Number of Buses = 82


In [161]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #pp acopf

1-HV-mixed--0-sw: Number of Buses = 306
1-HV-mixed--0-no_sw: Number of Buses = 64
1-HV-urban--0-sw: Number of Buses = 372
1-HV-urban--0-no_sw: Number of Buses = 82
1-MV-semiurb--0-sw: Number of Buses = 117
1-MV-semiurb--0-no_sw: Number of Buses = 115
1-MV-comm--0-sw: Number of Buses = 107
1-MV-comm--0-no_sw: Number of Buses = 103


In [9]:
net = sb.get_simbench_net('1-HV-mixed--0-no_sw')


In [6]:
dcopf_acopf_available_grid_names = ["1-HV-mixed--0-no_sw","1-HV-urban--0-no_sw", "1-MV-comm--0-no_sw", "1-MV-semiurb--0-no_sw"]

In [7]:
datasets_df_as_list = []
for grid_name in dcopf_acopf_available_grid_names:
    print("Generating datasets for grid " + grid_name)
    while len(datasets_df_as_list) != 100:
        net = sb.get_simbench_net(grid_name)
        df = create_dataset_from_dcopf_and_acopf(net)
        if df is not None:
            datasets_df_as_list.append(df)
            print(str(len(datasets_df_as_list)) + " Dataset(s) Generated" + " for the grid " + grid_name)

    print("Saving the datasets for grid " + grid_name)
    path = os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Supervised\\Training\\" + grid_name
    if not os.path.exists(path):
        os.makedirs(path)

    for i in range(0, len(datasets_df_as_list)):
        newpath = path + "\\Dataset-" + str(i) + ".csv"
        datasets_df_as_list[i].to_csv(newpath)

    datasets_df_as_list.clear()

Generating datasets for grid 1-HV-mixed--0-no_sw
0 Datasets Generated for the grid 1-HV-mixed--0-no_sw
1 Datasets Generated for the grid 1-HV-mixed--0-no_sw
2 Datasets Generated for the grid 1-HV-mixed--0-no_sw
3 Datasets Generated for the grid 1-HV-mixed--0-no_sw
4 Datasets Generated for the grid 1-HV-mixed--0-no_sw
5 Datasets Generated for the grid 1-HV-mixed--0-no_sw
6 Datasets Generated for the grid 1-HV-mixed--0-no_sw
7 Datasets Generated for the grid 1-HV-mixed--0-no_sw
8 Datasets Generated for the grid 1-HV-mixed--0-no_sw
9 Datasets Generated for the grid 1-HV-mixed--0-no_sw
10 Datasets Generated for the grid 1-HV-mixed--0-no_sw
11 Datasets Generated for the grid 1-HV-mixed--0-no_sw
12 Datasets Generated for the grid 1-HV-mixed--0-no_sw
13 Datasets Generated for the grid 1-HV-mixed--0-no_sw
14 Datasets Generated for the grid 1-HV-mixed--0-no_sw
15 Datasets Generated for the grid 1-HV-mixed--0-no_sw
16 Datasets Generated for the grid 1-HV-mixed--0-no_sw
17 Datasets Generated for 

In [15]:
dcopf_available_grid_names = ["1-HVMV-mixed-all-0-no_sw", "1-HVMV-mixed-1.105-0-no_sw", "1-HVMV-mixed-2.102-0-no_sw","1-HVMV-mixed-4.101-0-no_sw", "1-HVMV-urban-all-0-no_sw", "1-HVMV-urban-2.203-0-no_sw", "1-HVMV-urban-3.201-0-no_sw", "1-HVMV-urban-4.201-0-no_sw", "1-HV-mixed--0-no_sw", "1-HV-urban--0-no_sw", "1-MVLV-rural-all-0-no_sw"]

In [16]:
datasets_df_as_list = []
for grid_name in dcopf_available_grid_names:
    print("Generating datasets for grid " + grid_name)
    while len(datasets_df_as_list) != 100:
        net = sb.get_simbench_net(grid_name)
        df = create_dataset_from_dcopf(net)
        if df is not None:
            datasets_df_as_list.append(df)
            print(str(len(datasets_df_as_list)) + " Dataset(s) Generated" + " for the grid " + grid_name)

    print("Saving the datasets for grid " + grid_name)
    path = os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Unsupervised\\Training\\" + grid_name
    if not os.path.exists(path):
        os.makedirs(path)

    for i in range(0, len(datasets_df_as_list)):
        newpath = path + "\\Dataset-" + str(i) + ".csv"
        datasets_df_as_list[i].to_csv(newpath)

    datasets_df_as_list.clear()

Generating datasets for grid 1-HVMV-mixed-all-0-no_sw
1 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
2 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
3 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
4 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
5 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
6 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
7 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
8 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
9 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
10 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
11 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
12 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
13 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
14 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
15 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
16 Dataset(s) Generated f

In this revised version of the compute_node_embeddings function, the node features are weighted by the reverse admittance values in the adjacency matrix before they are summed to compute the node embeddings. The resulting node embeddings will reflect the strength of the connections between the nodes.

To use the reverse admittance values as the edge weights, you would need to pass the Ybus matrix as the adjacency matrix when calling the compute_node_embeddings function. The Ybus matrix should be converted to a PyTorch tensor before passing it to the function.

use the reverse admittance values as edge weights, you can modify the computation of the node embeddings to weight the node features by the reverse admittance values.

# Define the node types
node_types = ['Slack Node', 'Generator Node', 'Load Node']

# Define the number of nodes of each type in the graph
num_nodes = {
    'Slack Node': 1,
    'Generator Node': 20,
    'Load Node': 99
}


In [2]:
# Load datasets from local
grid_name = os.listdir(os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Supervised\\")[0]

#Get training, validation and test data including the edge index and weights from the grid
train_data, val_data, test_data, edge_index, edge_weights = read_supervised_training_data(grid_name)

Calculating edge index and edge weights for the grid 1-HV-mixed--0-no_sw ...
Reading all of the .csv files from the directory of 1-HV-mixed--0-no_sw ...
Processing Training Data for 1-HV-mixed--0-no_sw ...
Processing Validation Data for 1-HV-mixed--0-no_sw ...
Processing Test Data for 1-HV-mixed--0-no_sw ...
Processing complete.


In [3]:
# Create data loaders for our datasets; shuffle for training, not for validation
training_loader = DataLoader(train_data, batch_size=1, shuffle=True)
validation_loader = DataLoader(val_data, batch_size=1, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1, shuffle=True)

In [4]:
#train_data[20].validate(raise_on_error=True)
#training_data.is_undirected()
len(train_data[0].x)

64

In [4]:
in_channels = 4
hidden_channels = 186
out_channels = 4
activation = "tanh"
num_layers = 3
dropout = 0.1
jk = "last"
lr = 0.001
layer_type = "TransConv"
# torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels)
model = GNN(in_channels, hidden_channels, num_layers, out_channels, dropout=dropout, norm=torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels),jk=jk, layer_type=layer_type)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
for name, param in model.named_parameters():
  print(name)
  print(param.size())

param = sum(p.numel() for p in model.parameters() if p.requires_grad)
param

bias
torch.Size([])
convs.0.lin_key.weight
torch.Size([186, 4])
convs.0.lin_key.bias
torch.Size([186])
convs.0.lin_query.weight
torch.Size([186, 4])
convs.0.lin_query.bias
torch.Size([186])
convs.0.lin_value.weight
torch.Size([186, 4])
convs.0.lin_value.bias
torch.Size([186])
convs.0.lin_skip.weight
torch.Size([186, 4])
convs.0.lin_skip.bias
torch.Size([186])
convs.1.lin_key.weight
torch.Size([186, 186])
convs.1.lin_key.bias
torch.Size([186])
convs.1.lin_query.weight
torch.Size([186, 186])
convs.1.lin_query.bias
torch.Size([186])
convs.1.lin_value.weight
torch.Size([186, 186])
convs.1.lin_value.bias
torch.Size([186])
convs.1.lin_skip.weight
torch.Size([186, 186])
convs.1.lin_skip.bias
torch.Size([186])
convs.2.lin_key.weight
torch.Size([186, 186])
convs.2.lin_key.bias
torch.Size([186])
convs.2.lin_query.weight
torch.Size([186, 186])
convs.2.lin_query.bias
torch.Size([186])
convs.2.lin_value.weight
torch.Size([186, 186])
convs.2.lin_value.bias
torch.Size([186])
convs.2.lin_skip.weight
t

283837

In [19]:
for i in enumerate(model.modules()):
    print(i)
print("Shape of the output " + str(np.shape(model(train_data[0].x, edge_index, edge_attr=edge_weights))))

(0, GNN(4, 4, num_layers=3))
(1, Tanh())
(2, ModuleList(
  (0): TransformerConv(4, 186, heads=1)
  (1): TransformerConv(186, 186, heads=1)
  (2): TransformerConv(186, 186, heads=1)
))
(3, TransformerConv(4, 186, heads=1))
(4, SumAggregation())
(5, Linear(4, 186, bias=True))
(6, Linear(4, 186, bias=True))
(7, Linear(4, 186, bias=True))
(8, Linear(4, 186, bias=True))
(9, TransformerConv(186, 186, heads=1))
(10, SumAggregation())
(11, Linear(186, 186, bias=True))
(12, Linear(186, 186, bias=True))
(13, Linear(186, 186, bias=True))
(14, Linear(186, 186, bias=True))
(15, TransformerConv(186, 186, heads=1))
(16, SumAggregation())
(17, Linear(186, 186, bias=True))
(18, Linear(186, 186, bias=True))
(19, Linear(186, 186, bias=True))
(20, Linear(186, 186, bias=True))
(21, ModuleList(
  (0): BatchNorm(186)
  (1): BatchNorm(186)
  (2): BatchNorm(186)
))
(22, BatchNorm(186))
(23, BatchNorm1d(186, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
(24, BatchNorm(186))
(25, BatchNorm1d(1

In [6]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="OPF-GNN-Supervised-Homo",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "Homo-GNN",
    "num_layers": num_layers,
    "dataset": grid_name,
    "epochs": 400,
    "activation": activation,
    "dropout": dropout,
    "in_channels": in_channels,
    "output_channels": out_channels,
    "hidden_channels": hidden_channels,
    "channel type": layer_type,
    "norm": "BatchNorm"

    }
)


wandb: Currently logged in as: canbay96 (cbml). Use `wandb login --relogin` to force relogin


In [7]:
num_epochs = wandb.run.config.epochs
#MSE-Loss
loss = nn.MSELoss()
for i in range(num_epochs):
    #train_one_epoch(i, optimizer, training_loader, model, nn.MSELoss(), edge_index, edge_weights)
    train_validate_one_epoch(i, optimizer, training_loader, validation_loader, model, loss, edge_index, edge_weights)


Training the model for epoch 0
Validating the model on unseen Datasets for epoch 0
Training the model for epoch 1
Validating the model on unseen Datasets for epoch 1
Training the model for epoch 2
Validating the model on unseen Datasets for epoch 2
Training the model for epoch 3
Validating the model on unseen Datasets for epoch 3
Training the model for epoch 4
Validating the model on unseen Datasets for epoch 4
Training the model for epoch 5
Validating the model on unseen Datasets for epoch 5
Training the model for epoch 6
Validating the model on unseen Datasets for epoch 6
Training the model for epoch 7
Validating the model on unseen Datasets for epoch 7
Training the model for epoch 8
Validating the model on unseen Datasets for epoch 8
Training the model for epoch 9
Validating the model on unseen Datasets for epoch 9
Training the model for epoch 10
Validating the model on unseen Datasets for epoch 10
Training the model for epoch 11
Validating the model on unseen Datasets for epoch 11


In [8]:
# Test the model on unseen test datasets
output, target = test_one_epoch(test_loader, model, nn.MSELoss(), edge_index, edge_weights)

In [9]:
output

array([[ 1.09694588e+00,  2.13887615e+01,  1.75577148e+02,
        -2.17371655e+01],
       [ 1.09799981e+00,  9.62321758e-01,  7.13529724e+02,
        -3.25313660e+02],
       [ 1.09910583e+00,  1.98887997e+01, -4.49733353e+00,
        -8.03476944e+01],
       [ 1.06601596e+00,  2.10637894e+01, -6.01889992e+00,
        -6.32436931e-01],
       [ 1.08290708e+00,  2.07151947e+01,  1.33899088e+01,
         9.74170303e+00],
       [ 1.01891053e+00,  1.14831657e+01,  5.83778715e+00,
        -4.63461995e-01],
       [ 1.08759880e+00,  2.11373272e+01,  8.22590733e+00,
         1.17682743e+01],
       [ 1.08370578e+00,  1.98863277e+01, -2.40590153e+01,
        -1.43752813e+01],
       [ 1.08224738e+00,  2.08701515e+01,  3.09937429e+00,
         5.22535467e+00],
       [ 1.01595712e+00,  1.15862141e+01,  2.28606853e+01,
         4.79167366e+00],
       [ 1.08821261e+00,  2.23059235e+01, -1.47821762e+02,
        -3.31631966e+01],
       [ 1.02541614e+00,  1.28796511e+01, -1.49446621e+01,
      

In [10]:
target

array([[ 1.0997556e+00,  2.1400219e+01,  1.7739267e+02, -3.1531679e+01],
       [ 1.0920000e+00,  0.0000000e+00,  6.8882556e+02, -3.3112015e+02],
       [ 1.1000000e+00,  2.0600040e+01, -1.9999952e+01, -7.3410095e+01],
       [ 1.0675070e+00,  2.0869560e+01, -5.5936589e-06, -7.4907075e-06],
       [ 1.0848233e+00,  2.0723555e+01,  3.2267761e+00,  1.2958987e+00],
       [ 1.0187812e+00,  1.1286661e+01,  2.8991823e+00,  1.2056172e+00],
       [ 1.0866417e+00,  2.0416636e+01, -2.6363747e+00,  1.0932665e+00],
       [ 1.0860149e+00,  2.0608999e+01, -1.7978018e+01,  1.2161821e+00],
       [ 1.0842813e+00,  2.0885836e+01, -6.7557173e+00,  1.1975579e+00],
       [ 1.0161369e+00,  1.1306160e+01,  8.1866903e+00,  1.2474909e+01],
       [ 1.0925801e+00,  2.4067207e+01, -1.4906000e+02, -7.4907075e-06],
       [ 1.0305442e+00,  1.3563870e+01, -1.3138464e+01,  6.2285357e+00],
       [ 1.0318807e+00,  1.6199949e+01, -1.2805751e+02,  1.3577870e+01],
       [ 1.0233548e+00,  1.3917994e+01, -3.3109333e

In [23]:
len(training_loader)

85

In [11]:
scaler = MinMaxScaler()
scaler.fit_transform(training_loader.dataset[0].x, training_loader.dataset[0].y)

array([[0.        , 0.86117682, 0.36225094, 0.        ],
       [0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.6789371 , 0.20988076, 0.        ],
       [0.        , 0.7989771 , 0.17555885, 0.        ],
       [0.        , 0.70240025, 0.17940876, 0.05371227],
       [0.        , 0.3895996 , 0.17937463, 0.06062008],
       [0.        , 0.68925813, 0.17232448, 0.0523013 ],
       [0.        , 0.69747299, 0.15472186, 0.05403592],
       [0.        , 0.70958389, 0.16781204, 0.05677532],
       [0.        , 0.38888861, 0.18351918, 0.62377976],
       [0.        , 0.94940047, 0.        , 0.        ],
       [0.        , 0.46243351, 0.15917331, 0.34850452],
       [0.        , 0.56290248, 0.02410212, 0.60854019],
       [0.        , 0.48056316, 0.13669354, 0.06040978],
       [0.        , 0.58194992, 0.17555885, 0.        ],
       [0.        , 0.37836144, 0.22151934, 0.95421435],
       [0.        , 0.70896271, 0.15647893, 0.        ],
       [0.        , 0.56938017,